#### [1] 데이터 준비
- 데이터 정리 및 두 데이터셋 병합

[1-1] 데이터 정리
<hr>

In [1]:
import pandas as pd

In [3]:

df = pd.read_csv("./Data/human_made.csv", encoding="utf-8")

# 필요한 컬럼만 남기기 (Unnamed 같은 쓰레기 컬럼 제거)
df = df[["id", "text", "label"]]

# 완전 빈 행 제거
df = df.dropna(subset=["id", "text", "label"])

# 공백만 있는 행 제거
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"] != ""]

# label 정수화
df["label"] = df["label"].astype(int)

df.to_csv("./Data/human_made_clean.csv", index=False, encoding="utf-8")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb3 in position 0: invalid start byte

In [ ]:

df = pd.read_csv("./Data/AI_made.csv", encoding="utf-8")

# 필요한 컬럼만 남기기 (Unnamed 같은 쓰레기 컬럼 제거)
df = df[["id", "text", "label"]]

# 완전 빈 행 제거
df = df.dropna(subset=["id", "text", "label"])

# 공백만 있는 행 제거
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"] != ""]

# label 정수화
df["label"] = df["label"].astype(int)

df.to_csv("./Data/AI_made_clean.csv", index=False, encoding="utf-8")


In [ ]:
df = pd.read_csv('./Data/human_made_clean.csv', encoding='utf-8')
df.info()
print()
df = pd.read_csv('./Data/AI_made_clean.csv', encoding='utf-8')
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      30 non-null     float64
 1   text    30 non-null     str    
 2   label   30 non-null     int64  
dtypes: float64(1), int64(1), str(1)
memory usage: 852.0 bytes

<class 'pandas.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      30 non-null     int64
 1   text    30 non-null     str  
 2   label   30 non-null     int64
dtypes: int64(2), str(1)
memory usage: 852.0 bytes


[1-2] 데이터 병합 <hr>

In [ ]:
# =========================
# 설정: 입력/출력 경로
# =========================
HUMAN_PATH = "./Data/human_made_clean.csv"
AI_PATH    = "./Data/AI_made_clean.csv"
OUT_PATH   = "./Data/all.csv"

# =========================
# 1) CSV 로드
# =========================
human = pd.read_csv(HUMAN_PATH, encoding="utf-8")  # utf-8로 저장했으면 utf-8로 바꿔도 됨
ai    = pd.read_csv(AI_PATH, encoding="utf-8")

# 컬럼 체크 (id, text, label 존재해야 함)
need_cols = {"id", "text", "label"}
if not need_cols.issubset(human.columns) or not need_cols.issubset(ai.columns):
    raise ValueError(f"CSV 컬럼 확인 필요: 반드시 {need_cols} 포함해야 함")

# =========================
# 2) 라벨 규칙 통일
#   목표: 0=Human, 1=AI
#   현재 파일은 반대로 들어가 있어서 뒤집어줌
# =========================
human["label"] = 0  # 사람 데이터는 무조건 0
ai["label"] = 1     # AI 데이터는 무조건 1

# (선택) 출처 컬럼 추가하면 나중에 확인하기 좋음
human["source"] = "human"
ai["source"] = "ai"

# =========================
# 3) 텍스트 정리 (최소만)
# =========================
def clean_text(s):
    s = str(s)
    s = s.replace("\ufeff", "")  # BOM 제거
    s = s.strip()
    return s

human["text"] = human["text"].apply(clean_text)
ai["text"] = ai["text"].apply(clean_text)

# 빈 텍스트 제거
human = human[human["text"] != ""]
ai = ai[ai["text"] != ""]

# (선택) 너무 짧은 텍스트 제거 (예: 20자 미만)
MIN_CHARS = 20
human = human[human["text"].str.len() >= MIN_CHARS]
ai = ai[ai["text"].str.len() >= MIN_CHARS]

# =========================
# 4) 병합 + 중복 제거
# =========================
all_df = pd.concat([human, ai], ignore_index=True)

# 텍스트 완전 동일 중복 제거(중요)
all_df = all_df.drop_duplicates(subset=["text"]).reset_index(drop=True)

# id가 겹칠 수 있으니 새 id 부여(권장)
all_df.insert(0, "new_id", range(1, len(all_df) + 1))

# =========================
# 5) 저장
# =========================
all_df.to_csv(OUT_PATH, index=False, encoding="utf-8")

print("✅ 저장 완료:", OUT_PATH)
print("전체 샘플 수:", len(all_df))
print("라벨 분포:\n", all_df["label"].value_counts())
print("샘플 컬럼:", list(all_df.columns))
print(all_df.head(3))

✅ 저장 완료: ./Data/all.csv
전체 샘플 수: 60
라벨 분포:
 label
0    30
1    30
Name: count, dtype: int64
샘플 컬럼: ['new_id', 'id', 'text', 'label', 'source']
   new_id   id                                               text  label  \
0       1  1.0  너도 나도 창의성, 발상의 전환이 필요하다고 외칩니다. 하지만 주구장창 발상의 전환...      0   
1       2  2.0  다양한 현장을 넘나들고 취재한 경험을 통해 통합적 인재로 한발 나아갈 수 있었습니다...      0   
2       3  3.0  전략적 소통\n\n소통은 끊임없이 벌어집니다. 선의를 가진 사람과 소통할 경우엔 문...      0   

  source  
0  human  
1  human  
2  human  
